# **Assignment-2 for CS60075: Natural Language Processing**

#### Instructor : Prof. Sudeshna Sarkar

#### Teaching Assistants : Alapan Kuila, Aniruddha Roy, Prithwish Jana, Udit Dharmin Desai

#### Date of Announcement: 15th Sept, 2021
#### Deadline for Submission: 11.59pm on Wednesday, 22nd Sept, 2021 
#### Submit this .ipynb file, named as `<Your_Roll_Number>_Assn2_NLP_A21.ipynb`

The central idea of this assignment is to use Naive Bayes classifier and LSTM based classifier and compare the models by accuracy on IMDB dataset.  This dataset consists of 50k movie reviews (25k positive, 25k negative). You can download the dataset from https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews



Please submit with outputs. 

In [1]:
import re
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import keras
from sklearn.metrics import classification_report

In [2]:
filetrain=pd.read_csv("IMDB Dataset.csv")
col1=filetrain['review']
col2=filetrain['sentiment']
for i in range(2):
    print(col1[i]," ",col2[i])

One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fac

# Preprocessing
PrePrecessing that needs to be done on lower cased corpus

1. Remove html tags
2. Remove URLS
3. Remove non alphanumeric character
4. Remove Stopwords
5. Perform stemming and lemmatization

You can use regex from re. 

In [3]:
from bs4 import BeautifulSoup

text1=[]
for i in range(len(col1)):
    soup = BeautifulSoup(col1[i])
    text = soup.get_text()
    text1.append(text)         #Removed tags and urls
    
# for i in range(2):
#     print(text1[i])

In [4]:
import nltk
nltk.download('punkt') # For tokenizers
from nltk.tokenize import word_tokenize,sent_tokenize
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
words=[]
sum=0
stop_words=set(stopwords.words('english'))
for i in range(len(text1)):
    words.append(nltk.word_tokenize(text1[i]))
    words[i]=[word.lower() for word in words[i] if word.isalnum()]   #removed non alphanumeric
    words[i]=[word.lower() for word in words[i] if word not in stop_words]  #removed stopwords
    sum=sum+len(words[i])
    # print(len(words[i]))


In [6]:
sum2=0
words2=words
words3=[]
Max=0
lemmatizer = WordNetLemmatizer()
for i in range(len(words2)):
    tempstr=""
    for wd in range(len(words2[i])):
        words2[i][wd]=lemmatizer.lemmatize(words2[i][wd]) 
        tempstr=tempstr+' '+words2[i][wd]
    words3.append(tempstr)
    # words2[i]=set(words2[i])
    sum2=sum2+(len(words2[i]))       #lemmatization completed
    if(len(words2[i])>Max):
        Max=len(words2[i])



In [7]:
# Print Statistics of Data like avg length of sentence , proposition of data w.r.t class labels
# print(len(col1))
count=0
cnt=0
for i in col2:
    if(i=='positive'):
        count=count+1
    if(i=='negative'):
        cnt=cnt+1
print("Positive reviews = ",count,"\nNegative reviews = ",cnt)
print("average no. of words per review = ",sum/len(col2))
print("maximum number of words = ",Max)
# print("average no. of words after lemmatization = ",sum2/len(col2))
# print(col2.count("positive"))



Positive reviews =  25000 
Negative reviews =  25000
average no. of words per review =  113.89472
maximum number of words =  1404


# Naive Bayes classifier

In [8]:
# get reviews column from df
reviews = words3

# get labels column from df
labels = col2

In [9]:
# Use label encoder to encode labels. Convert to 0/1
encoder = LabelEncoder()
encoded_labels = encoder.fit_transform(labels)
print((encoded_labels))
print(encoder.classes_)

[1 1 1 ... 0 0 0]
['negative' 'positive']


In [10]:
# Split the data into train and test (80% - 20%). 
# Use stratify in train_test_split so that both train and test have similar ratio of positive and negative samples.

train_sentences, test_sentences, train_labels, test_labels=train_test_split(reviews,encoded_labels,test_size=0.2)
# print((train_labels[:5]),(test_labels[:5]))
# print(type(train_sentences))

Here there are two approaches possible for building vocabulary for the naive Bayes.
1. Take the whole data (train + test) to build the vocab. In this way while testing there is no word which will be out of vocabulary.
2. Take the train data to build vocab. In this case, some words from the test set may not be in vocab and hence one needs to perform smoothing so that one the probability term is not zero.
 
You are supposed to go by the 2nd approach.
 
Also building vocab by taking all words in the train set is memory intensive, hence you are required to build vocab by choosing the top 2000 - 3000 frequent words in the training corpus.

> $ P(x_i | w_j) = \frac{ N_{x_i,w_j}\, +\, \alpha }{ N_{w_j}\, +\, \alpha*d} $


$N_{x_i,w_j}$ : Number of times feature $x_i$ appears in samples of class $w_j$

$N_{w_j}$ : Total count of features in class $w_j$

$\alpha$ : Parameter for additive smoothing. Here consider $\alpha$ = 1

$d$ : Dimentionality of the feature vector  $x = [x_1,x_2,...,x_d]$. In our case its the vocab size.






In [11]:
from sklearn.feature_extraction.text import CountVectorizer
# Use Count vectorizer to get frequency of the words

# max_features parameter : If not None, build a vocabulary that only consider the top max_features ordered by term frequency across the corpus.

vec = CountVectorizer(max_features = 3000)
X = vec.fit_transform(train_sentences)
X=X.toarray()
# Y = vec.fit_transform(test_sentences)
print(((X[0])))
# print(type(vec.get_feature_names()))
vocab=vec.get_feature_names()
# print(vocab)


[0 0 0 ... 0 0 0]


In [12]:
# Use laplace smoothing for words in test set not present in vocab of train set
def lap():
    ans=1/6000
    return ans



In [13]:
# Build the model. Don't use the model from sklearn
pos_count=[0]*3000
neg_count=[0]*3000
for i in range(len(train_labels)):
    if(train_labels[i]==0):
        neg_count=neg_count+X[i]
    else:
        pos_count=pos_count+X[i]

print(pos_count)
print(neg_count)

[1619  322  163 ...  138  372  127]
[1734  334  132 ...  349 1105   99]


In [14]:
# Test the model on test set and report Accuracy

vec2=CountVectorizer(max_features=3000)
Y=vec2.fit_transform(test_sentences)
Y=Y.toarray()
testvocab=vec2.get_feature_names()
pos_prob=[0]*10000
neg_prob=[0]*10000
pred=[0]*10000
for i in range(len(test_sentences)):
    for j in range(len(testvocab)):
        if(Y[i][j]>0):
            if(testvocab[j] in vocab):
                pos_prob[i]=pos_prob[i]+(pos_count[vocab.index(testvocab[j])]+1)/6000
                neg_prob[i]=neg_prob[i]+(neg_count[vocab.index(testvocab[j])]+1)/6000
            else:
                pos_prob[i]=pos_prob[i]+1/6000
                neg_prob[i]=neg_prob[i]+1/6000
    if(pos_prob[i]>neg_prob[i]):
        pred[i]=1
    # if(pred[i]==test_labels[i]):
    #     print("Match")
    # else:
    #     print("Wrong")
    



In [15]:
score=0
for i in range(len(pred)):
    if(pred[i]==test_labels[i]):
        score=score+1
print("correct predictions = ",score)
print("wrong predictions = ",len(pred)-score)
print("Accuracy in percent = ", score/100)

correct predictions =  7246
wrong predictions =  2754
Accuracy in percent =  72.46


# *LSTM* based Classifier

Use the above train and test splits.

In [16]:
# Hyperparameters of the model
vocab_size = 3000 # choose based on statistics
oov_tok = '<OOK>'
embedding_dim = 100
max_length = 200# choose based on statistics, for example 150 to 200
padding_type='post'
trunc_type='post'

In [17]:
# tokenize sentences
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index

# convert train dataset to sequence and pad sequences
train_sequences = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_sequences, padding='post', maxlen=max_length)

# convert Test dataset to sequence and pad sequences
test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_sequences, padding='post', maxlen=max_length)

In [18]:
# model initialization
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

# compile model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# model summary
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 200, 100)          300000    
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               84480     
_________________________________________________________________
dense (Dense)                (None, 24)                3096      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 25        
Total params: 387,601
Trainable params: 387,601
Non-trainable params: 0
_________________________________________________________________


In [19]:
num_epochs = 5
history = model.fit(train_padded, train_labels, 
                    epochs=num_epochs, verbose=1, 
                    validation_split=0.1)

Epoch 1/5
1125/1125 [==============================] - 97s 84ms/step - loss: 0.3943 - accuracy: 0.8243 - val_loss: 0.3142 - val_accuracy: 0.8658
Epoch 2/5
1125/1125 [==============================] - 106s 94ms/step - loss: 0.2913 - accuracy: 0.8811 - val_loss: 0.3089 - val_accuracy: 0.8705
Epoch 3/5
1125/1125 [==============================] - 113s 100ms/step - loss: 0.2535 - accuracy: 0.8990 - val_loss: 0.3212 - val_accuracy: 0.8723
Epoch 4/5
1125/1125 [==============================] - 109s 97ms/step - loss: 0.2242 - accuracy: 0.9117 - val_loss: 0.3250 - val_accuracy: 0.8658
Epoch 5/5
1125/1125 [==============================] - 108s 96ms/step - loss: 0.1942 - accuracy: 0.9239 - val_loss: 0.3547 - val_accuracy: 0.8673


In [40]:
# Calculate accuracy on Test data

prediction = model.predict(test_padded)


# Get probabilities
prob=[]
for i in range(len(prediction)):
    val=prediction[i][0]
    prob.append(val)

# Get labels based on probability 1 if p>= 0.5 else 0
lstmlabels=[]
for i in range(len(prob)):
    if(prob[i]>=0.5):
        ans=1
    else:
        ans=0
    lstmlabels.append(ans)

# Accuracy : one can use classification_report from sklearn
target_names=['negative','positive']
print(classification_report(test_labels,lstmlabels,target_names=target_names))


              precision    recall  f1-score   support

    negative       0.88      0.87      0.87      5045
    positive       0.87      0.88      0.87      4955

    accuracy                           0.87     10000
   macro avg       0.87      0.87      0.87     10000
weighted avg       0.87      0.87      0.87     10000



## Get predictions for random examples

In [42]:
# reviews on which we need to predict
sentence = ["The movie was very touching and heart whelming", 
            "I have never seen a terrible movie like this", 
            "the movie plot is terrible but it had good acting"]

# convert to a sequence
sequences = tokenizer.texts_to_sequences(sentence)

# pad the sequence
padded = pad_sequences(sequences, padding='post', maxlen=max_length)

# Get probabilities
print("Probabilities\n")
randomprob=model.predict(padded)
print(randomprob)
randomlabels=[]

# Get labels based on probability 1 if p>= 0.5 else 0
for i in range(3):
    if(randomprob[i][0]>=0.5):
        randomlabels.append(1)
    else:
        randomlabels.append(0)
for i in range(len(randomlabels)):
    if(randomlabels[i]==1):
        print('positive')
    else:
        print('negative')


Probabilities

[[0.94524586]
 [0.13283944]
 [0.13581777]]
positive
negative
negative
